### 1.游标cursor概念

#### 1.1游标类别
>搜索游标：arcpy.da.SearchCursor<br>
>插入游标：arcpy.da.InsertCursor<br>
>更新游标：arcpy.da.UpdataCursor<br>

In [1]:
import arcpy

In [11]:
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

#### 1.2搜索游标arcpy.da.SearchCursor

In [2]:
#直接进行数据查询，查询完毕后删除游标
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'XZQ_town'
#过滤出字符串的列
fields = arcpy.ListFields(fc,field_type="string")
#列出列名和属性
for field in fields:
    print("{0} is a type of {1} with a length of {2}"
          .format(field.name, field.type, field.length))
#设置想要查看的列
u_field = ['SM','A_belong_town']
#实例化游标
cursor = arcpy.da.SearchCursor(fc,u_field )
#打印列内容，注意u编码
for row in cursor:
    print(u'{0}, {1}'.format(row[0], row[1]))
del cursor

[u'XZQ_town']

In [35]:
#使用with语句进行数据查询
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'XZQ_town'
fields = ['OBJECTID','A_belong_town']
#不使用字符串u编码或报错，加encode('utf-8')一样的效果
with arcpy.da.SearchCursor(fc,fields) as cursor:
    for row in cursor:
        print('{0},{1}'.format(row[0],row[1].encode('utf-8')))

1,仙女镇
2,宜陵镇
3,吴桥镇
4,邵伯镇
5,丁伙镇
6,武坚镇
7,丁沟镇
8,大桥镇
9,真武镇
10,小纪镇
11,樊川镇
12,浦头镇
13,郭村镇


#### 1.3插入游标arcpy.da.InsertCursor

In [58]:
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'dot'
fields = ['city_name','SHAPE@XY']
#要插入的信息
row_values = [('chengdu',(104,30)),
              ('suzhou',(120,31)),
              ('nanjing',(118,32))]
with arcpy.da.InsertCursor(fc,fields) as cursor:
    for row in row_values:
        print(row)
        cursor.insertRow(row)

('chengdu', (104, 30))
('suzhou', (120, 31))
('nanjing', (118, 32))


#### 1.4更新游标arcpy.da.UpdataCursor

In [66]:
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'XZQ_town_update'
field = ['A_belong_town']

with arcpy.da.UpdateCursor(fc,field) as cursor:
    for row in cursor:
        if u'镇' not in row[0]:
            row[0] = row[0] + u'镇'
        #更新游标
        cursor.updateRow(row)
print('done')

done


In [68]:
#列出表对应的列名
arcpy.ListFields('XZQ_town_update','*town*')[0].name.encode('utf-8')

'A_belong_town'

In [77]:
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'XZQ_town_update_1'
field = ['SHAPE@AREA']

with arcpy.da.UpdateCursor(fc,field) as cursor:
    for row in cursor:
        if row[0] < 500000:
            #删除游标
            cursor.deleteRow()
print('done')

done


[u'XZQ_town', u'XZQ_vill_dt', u'dot', u'XZQ_town_update', u'XZQ_town_update_1']

#### 2.1使用令牌，获取几何对象信息shape@AREA shape@XY等

In [37]:
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'XZQ_town'
fields = ['OBJECTID','A_belong_town','shape@AREA','shape@XY']

with arcpy.da.SearchCursor(fc,fields) as cursor:
    for row in cursor:
        print(u'{},{},{},{}'.format(row[0],row[1],row[2],row[3]))

1,仙女镇,145529721.752,(40561623.53532807, 3539633.0581835127)
2,宜陵镇,59858689.529,(40569848.18776498, 3545419.723753655)
3,吴桥镇,55979141.5286,(40575840.03571186, 3540920.7188733243)
4,邵伯镇,122921115.329,(40553624.43162427, 3555642.1228415915)
5,丁伙镇,80910494.1719,(40563611.79783072, 3548694.578762301)
6,武坚镇,85671668.1256,(40581925.86031338, 3573359.5933980327)
7,丁沟镇,99940151.0661,(40571986.98141155, 3553918.960162455)
8,大桥镇,160161405.963,(40575264.13863847, 3532034.902787635)
9,真武镇,76047167.4298,(40559678.388640985, 3558956.5183717767)
10,小纪镇,176683230.755,(40579812.214309864, 3563827.1079171062)
11,樊川镇,118587550.981,(40568025.48309077, 3564656.523279125)
12,浦头镇,43147895.7467,(40583311.92192002, 3535875.528737229)
13,郭村镇,104495286.957,(40580348.892011866, 3547416.3676366312)


#### 2.2构建查询表达式，筛选对象

In [53]:
import arcpy
arcpy.env.workspace = r'D:\rui\code_analysis\homework\file\GIS\class2002.gdb'

fc = 'XZQ_vill_dt'
fields = ['OBJECTID','XZQMC','shape@AREA']
#查询表达式,外面是双引号
expression = "A_belong_town = '樊川镇'"

with arcpy.da.SearchCursor(fc,fields,where_clause=expression) as cursor:
    for row in cursor:
        print(row[1])

圣禾村
友爱村
直属村
樊南村
陆巷村
东汇村
五和村
联丰村
跃进村
和丰村
关河村
永新村
永安村
永联村
新圩村
聚永村
西闫村
联谊村
三周村
同丰村
大同村
